<a href="https://colab.research.google.com/github/FrancisKurian/CS530/blob/main/Linear_Regression_prediction_with_scipy_optimize_minimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contents organization
## Training Dataset
* Read data file student-por.csv, create a 80-20 split for Training and Test (keep G2 & G3 out)
* Use Statsmodel for linear regression(OLS) and extract coefficients , calculate MSE
* Set-up SciPy and generate optimized coefficients for the training data set.
* Compare MSE between training and SciPy Minimize

## Test Dataset
* Rescore the test dataset on training model, compare MSE with Training
* Use SciPy optmizer on test data, compare MSE with StatsModel MSE

## Change training & Test sample Size - 50%, 70% etc.
## Try different SciPy Methods -BFGS & L-BFGS-B  etc

#Reading Data/Dummy variable creation

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving student-por.csv to student-por.csv


In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('student-por.csv', sep = ";")
df = df.drop(columns=["Mjob",	"Fjob",	"reason",	"guardian"])

In [ ]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [ ]:
#cols= ['G3','G1','school',	'sex', 'address',	'famsize','Pstatus','Medu','Fedu','traveltime','studytime','failures','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic','famrel','freetime','goout','Dalc','Walc','health','absences']
cols= ['G3','G2','G1']
# cols= ['G3']
X=df.drop(columns=cols)
y=df['G3']
X = pd.get_dummies(X)
X.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,1,3,4,1,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0,1,0
1,17,1,1,1,2,0,5,3,3,1,1,3,2,1,0,1,0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0
2,15,1,1,1,2,0,4,3,2,2,3,3,6,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,1,1,0
3,15,4,2,1,3,0,3,2,2,1,1,5,0,1,0,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1
4,16,3,3,1,2,0,4,3,2,1,2,5,0,1,0,1,0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0


# Training/Test Splits

In [ ]:
# Sperate train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear Regression Model and Results

In [ ]:
import statsmodels.api as sm
ols_model=sm.OLS(y_train,X_train).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     10.53
Date:                Mon, 21 Feb 2022   Prob (F-statistic):           3.64e-33
Time:                        20:33:47   Log-Likelihood:                -1249.4
No. Observations:                 519   AIC:                             2553.
Df Residuals:                     492   BIC:                             2668.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
age                0.1851      0.114      1.

In [ ]:
from statsmodels.tools.eval_measures import rmse, mse
ypred = ols_model.predict(X_train)
# calc rmse
rmse = rmse(y_train, ypred)
mse = mse(y_train, ypred)
print("MSE:",mse,"RMSE:",rmse)

MSE: 7.218414563218563 RMSE: 2.686710733074657


# Extract Coefficients

In [ ]:
# extract coefficients from training model to an array for ScyPy function accuracy check
c_v1=ols_model.params.to_numpy()

#intial condition set to 1 for SciPy optimization
lst = []
for i in range(len(c_v1)):
    lst.append(1)
c_v2=np.array(lst)

In [ ]:
print(len(c_v1),len(c_v2))
print(c_v1,c_v2)
print(type(c_v1),type(c_v2))


39 39
[ 0.18508929  0.31722224  0.01262297  0.01002361  0.34518337 -1.46571039
  0.14631192 -0.11886066 -0.10470331 -0.25165689 -0.13164858 -0.14406395
 -0.01776317  1.12986527 -0.0194846   0.87554043  0.23484023  0.40115881
  0.70922186  0.40643425  0.70394642  0.52056594  0.58981473  1.15250971
 -0.04212904  0.59080374  0.51957693  0.81009338  0.30028729  0.35853319
  0.75184748  0.71706369  0.39331698 -0.42466806  1.53504873  0.31218198
  0.79819869  0.77595277  0.3344279 ] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
# Extarct X variables as array to automate SciPy
Xn2=[]
for col in X_train.columns:
    Xn2.append(X_train[col].to_numpy())
Y= y_train.to_numpy()

#SciPy Set-up
## functions to compute models and MSE

In [ ]:
def model(coeffs, X):
    f= np.sum(X*coeffs[:,np.newaxis],axis=0)
    return f

def sum_of_squares(coeff, X, Y):
    f = model(coeff, X)
    obj = np.array(((f - Y)**2)).sum()
    return obj

## Optimize with initial conditions of the coefficients set

In [ ]:
from scipy.optimize import minimize
res = minimize(sum_of_squares, c_v2, args=(Xn2, Y),method='L-BFGS-B')
print(res)

      fun: 3746.3573399421366
 hess_inv: <39x39 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.11905286, -0.07908056, -0.05734364, -0.00100044,  0.07880772,
        0.02551133, -0.01559783,  0.05566108, -0.05893526, -0.20172592,
       -0.21764208,  0.02455636, -0.20113475,  0.0428372 , -0.05720722,
       -0.00754881, -0.00677574, -0.00518412, -0.00914042, -0.02683009,
        0.01246008,  0.02260094, -0.03697096, -0.05475158,  0.04038156,
       -0.00318323, -0.01109584,  0.08681127, -0.10113581,  0.05275069,
       -0.06707523, -0.01837179,  0.00409273, -0.03346941,  0.01919034,
        0.02346496, -0.03783498,  0.04956746, -0.06384653])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 4440
      nit: 98
   status: 0
  success: True
        x: array([ 0.18525625,  0.31720463,  0.0126459 ,  0.01006251,  0.34532219,
       -1.46556826,  0.14628641, -0.11872778, -0.10468251, -0.25181886,
       -0.13168873, -0.14403902, -0.01776723,  1.12968399, -0

In [ ]:
#extract optimized coefficients
c_v3=res.x
print(c_v3)

[ 0.18525625  0.31720463  0.0126459   0.01006251  0.34532219 -1.46556826
  0.14628641 -0.11872778 -0.10468251 -0.25181886 -0.13168873 -0.14403902
 -0.01776723  1.12968399 -0.01993909  0.87507253  0.23467171  0.40092265
  0.70882137  0.4061112   0.70363478  0.52033099  0.58941165  1.15199836
 -0.04225346  0.59046817  0.51927698  0.81073435  0.29901097  0.35832765
  0.75141602  0.71671737  0.39302751 -0.42521339  1.53495967  0.31191603
  0.79783124  0.77574949  0.33399532]


#MSE using optmized coefficients

In [ ]:
sum_of_squares(c_v3, Xn2,Y)/len(Y)

7.218414913183308

# Regression: Training Model, Test Data 

In [ ]:
from statsmodels.tools.eval_measures import rmse, mse
ypred = ols_model.predict(X_test)
# calc rmse
rmse = rmse(y_test, ypred)
mse = mse(y_test, ypred)
print("MSE:",mse,"RMSE:",rmse)

MSE: 5.627686979702675 RMSE: 2.3722746425535712


# Function to List MSE for various sample size, optimization techniques

In [ ]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse, mse
from scipy.optimize import minimize

def Ols_scenarios(Test_Size, SciPyMethod):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=Test_Size, random_state=0)
  ols_model=sm.OLS(y_train,X_train).fit()
  ypred_tr = ols_model.predict(X_train)
  mse_tr = mse(y_train, ypred_tr)

  ypred_t = ols_model.predict(X_test)
  mse_t = mse(y_test, ypred_t)

  c_v1=ols_model.params.to_numpy()
  lst = []
  for i in range(len(c_v1)):
    lst.append(1)
  c_v2=np.array(lst)
 

  Xn2=[]
  Xn3=[]
  for col in X_train.columns:
    Xn2.append(X_train[col].to_numpy())
    Yn2= y_train.to_numpy()
    Xn3.append(X_test[col].to_numpy())
    Yn3= y_test.to_numpy()  
  res = minimize(sum_of_squares, c_v2, args=(Xn2, Yn2), options={'disp':True},method=SciPyMethod)
  c_v3=res.x #optimized coef array
  mse_o_tr=sum_of_squares(c_v3, Xn2,Yn2)/len(Yn2)
  mse_o_t=sum_of_squares(c_v3, Xn3,Yn3)/len(Yn3)
  print( 'MSE-Training:',mse_tr)
  print( 'MSE-Test:',mse_t)
  print( 'MSE-Train-Optimized:(',SciPyMethod,')',mse_o_tr)
  print( 'MSE-Test-Optimized:(',SciPyMethod,')',mse_o_t)

  return   mse_tr, mse_t, mse_o_tr,mse_o_t

In [ ]:
cols= ['G3']
X=df.drop(columns=cols)
X = pd.get_dummies(X)
y=df['G3']

print("Training 80%, Test 20%, Optimization Method:L-BFGS-B")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.2,"L-BFGS-B")

print("Training 50%, Test 50%, Optimization Method:L-BFGS-B")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.5,"L-BFGS-B")

print("Training 80%, Test 20%, Optimization Method: BFGS")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.2,"BFGS")

print("Training 50%, Test 50%, Optimization Method: BFGS")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.5,"BFGS")

Training 80%, Test 20%, Optimization Method:L-BFGS-B
MSE-Training: 1.636137114720974
MSE-Test: 1.0236201523551807
MSE-Train-Optimized:( L-BFGS-B ) 1.63613730182413
MSE-Test-Optimized:( L-BFGS-B ) 1.0236483283085633
Training 50%, Test 50%, Optimization Method:L-BFGS-B
MSE-Training: 1.2954053642393621
MSE-Test: 1.9323417045414921
MSE-Train-Optimized:( L-BFGS-B ) 1.2954055976842664
MSE-Test-Optimized:( L-BFGS-B ) 1.9324172094920589
Training 80%, Test 20%, Optimization Method: BFGS
         Current function value: 849.155163
         Iterations: 39
         Function evaluations: 2538
         Gradient evaluations: 59
MSE-Training: 1.636137114720974
MSE-Test: 1.0236201523551807
MSE-Train-Optimized:( BFGS ) 1.6361371147240868
MSE-Test-Optimized:( BFGS ) 1.0236201893955152
Training 50%, Test 50%, Optimization Method: BFGS
         Current function value: 419.711338
         Iterations: 41
         Function evaluations: 2538
         Gradient evaluations: 59
MSE-Training: 1.2954053642393621
MS

In [ ]:
# Add G2 and G1 Back
cols= ['G3','G2','G1']
X=df.drop(columns=cols)
X = pd.get_dummies(X)

print("Training 80%, Test 20%, Optimization Method:L-BFGS-B")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.2,"L-BFGS-B")

print("Training 50%, Test 50%, Optimization Method:L-BFGS-B")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.5,"L-BFGS-B")

print("Training 80%, Test 20%, Optimization Method: BFGS")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.2,"BFGS")

print("Training 50%, Test 50%, Optimization Method: BFGS")
mse_tr, mse_t, mse_o_tr,mse_o_t=Ols_scenarios(0.5,"BFGS")


Training 80%, Test 20%, Optimization Method:L-BFGS-B
MSE-Training: 7.218414563218563
MSE-Test: 5.627686979702675
MSE-Train-Optimized:( L-BFGS-B ) 7.218414913183308
MSE-Test-Optimized:( L-BFGS-B ) 5.627499419104615
Training 50%, Test 50%, Optimization Method:L-BFGS-B
MSE-Training: 5.7048815776039055
MSE-Test: 8.493869884936045
MSE-Train-Optimized:( L-BFGS-B ) 5.704881681825842
MSE-Test-Optimized:( L-BFGS-B ) 8.49378460972242
Training 80%, Test 20%, Optimization Method: BFGS
         Current function value: 3746.357158
         Iterations: 51
         Function evaluations: 3076
         Gradient evaluations: 75
MSE-Training: 7.218414563218563
MSE-Test: 5.627686979702675
MSE-Train-Optimized:( BFGS ) 7.218414563221563
MSE-Test-Optimized:( BFGS ) 5.627686909056814
Training 50%, Test 50%, Optimization Method: BFGS
         Current function value: 1848.381631
         Iterations: 36
         Function evaluations: 2133
         Gradient evaluations: 52
MSE-Training: 5.7048815776039055
MSE-Test

### End

@ Jeomoan Francis Kurian